In [191]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split


In [192]:
# データの読み込み
data = pd.read_csv('../../data/preprocessed_data.csv')

# 特徴量とターゲット変数の分割,DFのvalueのみ抽出
X = data.drop('dengue', axis=1).values  
y = data['dengue'].values

In [193]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [194]:
#numpy配列をtensolに変換
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [195]:
#TensorDatasetの作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [196]:
# ニューラルネットワークの定義
class ClassificationNN(nn.Module):
    def __init__(self, input_size):
        super(ClassificationNN, self).__init__()
        self.fc1 = nn.Linear(input_size,64)  
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)           
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))         
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))      
        return x


In [200]:
# モデルの初期化
input_size = X_train_tensor.shape[1]  
model = ClassificationNN(input_size)

# 損失関数と最適化手法の設定
criterion = nn.BCELoss()  # バイナリクロスエントロピー損失関数
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [204]:
num_epochs = 2000
for epoch in range(num_epochs):
    model.train()  # 訓練モード

    # 順伝播
    outputs = model(X_train_tensor)

    # 損失計算
    loss = criterion(outputs, y_train_tensor)
    
    # 勾配の初期化
    optimizer.zero_grad()
    
    # 逆伝播とパラメータ更新
    loss.backward()
    optimizer.step()

    # 精度（accuracy）の計算
    predicted = outputs.round()  # 0.5以上の予測値は1、それ以外は0に丸める
    correct = (predicted == y_train_tensor).float().sum().item()
    accuracy = correct / y_train_tensor.size(0) * 100  # 全サンプル数に対する正解率

    # 10エポックごとに損失と精度を表示
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')


Epoch [10/2000], Loss: 0.1385, Accuracy: 94.23%
Epoch [20/2000], Loss: 0.1382, Accuracy: 94.23%
Epoch [30/2000], Loss: 0.1380, Accuracy: 94.23%
Epoch [40/2000], Loss: 0.1379, Accuracy: 94.30%
Epoch [50/2000], Loss: 0.1374, Accuracy: 94.23%
Epoch [60/2000], Loss: 0.1370, Accuracy: 94.30%
Epoch [70/2000], Loss: 0.1366, Accuracy: 94.23%
Epoch [80/2000], Loss: 0.1363, Accuracy: 94.23%
Epoch [90/2000], Loss: 0.1361, Accuracy: 94.30%
Epoch [100/2000], Loss: 0.1359, Accuracy: 94.27%
Epoch [110/2000], Loss: 0.1357, Accuracy: 94.30%
Epoch [120/2000], Loss: 0.1354, Accuracy: 94.27%
Epoch [130/2000], Loss: 0.1352, Accuracy: 94.30%
Epoch [140/2000], Loss: 0.1350, Accuracy: 94.27%
Epoch [150/2000], Loss: 0.1345, Accuracy: 94.30%
Epoch [160/2000], Loss: 0.1341, Accuracy: 94.33%
Epoch [170/2000], Loss: 0.1340, Accuracy: 94.30%
Epoch [180/2000], Loss: 0.1336, Accuracy: 94.33%
Epoch [190/2000], Loss: 0.1333, Accuracy: 94.30%
Epoch [200/2000], Loss: 0.1331, Accuracy: 94.33%
Epoch [210/2000], Loss: 0.132

In [189]:
#バッチ処理番

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()  # 勾配の初期化

        # バッチデータを使って順伝播
        outputs = model(X_batch)
        
        # 損失計算
        loss = criterion(outputs, Y_batch)
        
        # 逆伝播とパラメータの更新
        loss.backward()
        optimizer.step()
        
        # 累積損失の計算
        running_loss += loss.item()
        
        # 精度（accuracy）の計算
        predicted = outputs.round()  # 0.5以上は1、未満は0に丸める
        correct += (predicted == Y_batch).sum().item()
        total += Y_batch.size(0)

    # エポックごとの損失と精度を表示
    epoch_loss = running_loss / len(train_loader)
    accuracy = correct / total * 100
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')


Epoch [1/100], Loss: 0.6884, Accuracy: 55.70%
Epoch [2/100], Loss: 0.6762, Accuracy: 58.01%
Epoch [3/100], Loss: 0.6616, Accuracy: 60.55%
Epoch [4/100], Loss: 0.6456, Accuracy: 62.67%
Epoch [5/100], Loss: 0.6314, Accuracy: 63.81%
Epoch [6/100], Loss: 0.6204, Accuracy: 63.94%
Epoch [7/100], Loss: 0.6120, Accuracy: 64.36%
Epoch [8/100], Loss: 0.6053, Accuracy: 64.10%
Epoch [9/100], Loss: 0.5990, Accuracy: 64.14%
Epoch [10/100], Loss: 0.5930, Accuracy: 64.56%
Epoch [11/100], Loss: 0.5880, Accuracy: 64.63%
Epoch [12/100], Loss: 0.5821, Accuracy: 64.36%
Epoch [13/100], Loss: 0.5775, Accuracy: 64.56%
Epoch [14/100], Loss: 0.5729, Accuracy: 65.21%
Epoch [15/100], Loss: 0.5688, Accuracy: 65.37%
Epoch [16/100], Loss: 0.5653, Accuracy: 65.15%
Epoch [17/100], Loss: 0.5618, Accuracy: 65.37%
Epoch [18/100], Loss: 0.5588, Accuracy: 65.80%
Epoch [19/100], Loss: 0.5559, Accuracy: 66.12%
Epoch [20/100], Loss: 0.5534, Accuracy: 65.96%
Epoch [21/100], Loss: 0.5513, Accuracy: 66.87%
Epoch [22/100], Loss: 

In [190]:
# テストデータでの評価
model.eval()  # 評価モード
with torch.no_grad():
    predictions = model(X_test_tensor)
    predictions = predictions.round()  # 出力を0または1に丸める
    accuracy = (predictions == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.6436
